In [61]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as st

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost import plot_importance, XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from mlxtend.evaluate import bias_variance_decomp

from hyperopt import fmin, tpe, hp, STATUS_OK

In [67]:
train = pd.read_csv('data/train_final.csv', index_col = "Id")
test = pd.read_csv('data/test_final.csv', index_col = "Id")
#col 84 and 86 is float64, rest are int64
#col 0 is Id, col 1 is Y
#test has more rows than train
#col 2 and 3 have -999
#34, 39 are all zero in train

X, y = train.drop('Y', axis=1), train['Y']
test.describe()

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86
count,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.0,3854.000000,3854.000000,3854.000000,3854.000000,3854.0,3854.0,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000,3854.000000
mean,-0.497924,-3.262065,0.004930,0.425791,0.031136,0.002335,0.034510,0.030358,0.007265,0.016866,0.000259,0.210171,0.410742,0.042034,0.452517,0.009600,0.011417,0.901920,0.520498,0.090815,0.621432,0.006227,0.000778,0.013492,0.148158,0.621432,0.054229,0.279969,0.033991,0.012974,0.792423,0.0,0.000259,0.006487,0.052413,0.015828,0.0,0.0,0.316035,0.270109,0.760509,0.022833,0.005189,0.026466,0.368708,0.026725,0.009341,0.014530,0.003114,0.917488,0.955371,0.466788,0.016606,0.031655,0.005189,0.063830,0.005189,0.958225,0.346653,0.060976,0.044629,0.002335,0.955631,0.716917,0.234821,0.073690,0.035029,0.999481,0.012195,0.001297,0.052154,0.000778,0.205241,0.047483,0.416710,0.021536,0.194084,0.072133,0.004930,0.000259,0.044629,1.050337,4210.574468,0.035547,0.242727
std,22.755459,57.937152,0.070049,0.494527,0.173709,0.048274,0.182558,0.171593,0.084937,0.128785,0.016108,0.407483,0.492032,0.200693,0.497805,0.097523,0.106251,0.297461,0.499644,0.287383,0.485093,0.078677,0.027893,0.115386,0.355302,0.485093,0.226499,0.449042,0.181229,0.113175,0.405624,0.0,0.016108,0.080289,0.222887,0.124825,0.0,0.0,0.464987,0.444074,0.426828,0.149392,0.071860,0.160537,0.482517,0.161301,0.096209,0.119678,0.055720,0.275178,0.206515,0.498960,0.127807,0.175104,0.071860,0.244481,0.071860,0.200100,0.475966,0.239317,0.206515,0.048274,0.205941,0.450555,0.423942,0.261299,0.183876,0.022777,0.109770,0.036000,0.222366,0.027893,0.403930,0.212697,0.493078,0.145182,0.395545,0.258741,0.070049,0.016108,0.206515,0.290982,2429.397979,0.232417,1.892194
min,-999.000000,-999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.000000,0.000000,-6.523543
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0

In [65]:
# Replace -999 with mean
X = X.replace(-999, np.nan)
test = test.replace(-999, np.nan)
X = X.fillna(X.mean())
test = test.fillna(test.mean())

In [51]:
#inital testing with KNN, Logistic Regression, Random Forest, Linear SVC, SVC, Naive Bayes, Decision Tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#KNN
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
#bias and variance tradeoff
_, bias, variance = bias_variance_decomp(knn, X_train.values, y_train.values, X_test.values, y_test.values, loss='0-1_loss', num_rounds=200, random_seed=42)
print('KNN Variance:', variance)
# print('KNN AUC:', roc_auc_score(y_test, y_pred))

#Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
_, bias, variance = bias_variance_decomp(logreg, X_train.values, y_train.values, X_test.values, y_test.values, loss='0-1_loss', num_rounds=200, random_seed=42)
print('Logistic Regression Variance:', variance)
# print('Logistic Regression AUC:', roc_auc_score(y_test, y_pred))

#Linear SVC
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
y_pred = lsvc.predict(X_test)
_, bias, variance = bias_variance_decomp(lsvc, X_train.values, y_train.values, X_test.values, y_test.values, loss='0-1_loss', num_rounds=200, random_seed=42)
print('Linear SVC Variance:', variance)
# print('Linear SVC AUC:', roc_auc_score(y_test, y_pred))

#Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
_, bias, variance = bias_variance_decomp(nb, X_train.values, y_train.values, X_test.values, y_test.values, loss='0-1_loss', num_rounds=200, random_seed=42)
print('Naive Bayes Variance:', variance)
# print('Naive Bayes AUC:', roc_auc_score(y_test, y_pred))

#Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
_, bias, variance = bias_variance_decomp(dt, X_train.values, y_train.values, X_test.values, y_test.values, loss='0-1_loss', num_rounds=200, random_seed=42)
print('Decision Tree Variance:', variance)
# print('Decision Tree AUC:', roc_auc_score(y_test, y_pred))





KNN Variance: 0.2444746059544658
Logistic Regression Variance: 0.035166374781085814
Linear SVC Variance: 0.23365148861646234
Naive Bayes Variance: 0.054106830122591945
Decision Tree Variance: 0.1431786339754816


In [68]:
xgb_classifier = xgb.XGBClassifier()
xgb_cv_scores = cross_val_score(xgb_classifier, X, y, cv=5, scoring='roc_auc')

rf_classifier = RandomForestClassifier()
rf_cv_scores = cross_val_score(rf_classifier, X, y, cv=5, scoring='roc_auc')

cb_classifier = CatBoostClassifier()
cb_cv_scores = cross_val_score(cb_classifier, X, y, cv=5, scoring='roc_auc')

et_classifier = ExtraTreesClassifier()
et_cv_scores = cross_val_score(et_classifier, X, y, cv=5, scoring='roc_auc')

gb_classifier = GradientBoostingClassifier()
gb_cv_scores = cross_val_score(gb_classifier, X, y, cv=5, scoring='roc_auc')

lgb_classifier = lgb.LGBMClassifier()
lgb_cv_scores = cross_val_score(lgb_classifier, X, y, cv=5, scoring='roc_auc')

print('XGB AUC:', xgb_cv_scores.mean())
print('RF AUC:', rf_cv_scores.mean())
print('CB AUC:', cb_cv_scores.mean())
print('ET AUC:', et_cv_scores.mean())
print('GB AUC:', gb_cv_scores.mean())
print('LGB AUC:', lgb_cv_scores.mean())

Learning rate set to 0.014653
0:	learn: 0.6834373	total: 3.23ms	remaining: 3.23s
1:	learn: 0.6743669	total: 7.66ms	remaining: 3.82s
2:	learn: 0.6653656	total: 9.89ms	remaining: 3.29s
3:	learn: 0.6568465	total: 12.1ms	remaining: 3.02s
4:	learn: 0.6485987	total: 14.5ms	remaining: 2.88s
5:	learn: 0.6414329	total: 22.5ms	remaining: 3.73s
6:	learn: 0.6339722	total: 28.1ms	remaining: 3.98s
7:	learn: 0.6262062	total: 30.7ms	remaining: 3.8s
8:	learn: 0.6184748	total: 32.8ms	remaining: 3.61s
9:	learn: 0.6111096	total: 35.9ms	remaining: 3.55s
10:	learn: 0.6040373	total: 38.8ms	remaining: 3.49s
11:	learn: 0.5973545	total: 41.4ms	remaining: 3.41s
12:	learn: 0.5905559	total: 44.1ms	remaining: 3.35s
13:	learn: 0.5845301	total: 46.1ms	remaining: 3.25s
14:	learn: 0.5785751	total: 47.9ms	remaining: 3.14s
15:	learn: 0.5725992	total: 49.8ms	remaining: 3.06s
16:	learn: 0.5673771	total: 51.8ms	remaining: 3s
17:	learn: 0.5619062	total: 53.7ms	remaining: 2.93s
18:	learn: 0.5562961	total: 55.6ms	remaining: 2.

In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the training data
train_data = pd.read_csv('data/train_final.csv')
train_data.drop(columns=['Id'], inplace=True)
X_train = train_data.drop(columns=['Y'])
y_train = train_data['Y']

# Normalize the features and handle missing values
X_train = X_train.replace(-999, np.nan)
X_train = X_train.fillna(X.mean())

# Convert to PyTorch tensor
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)


# Define the BinaryClassifier
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(BinaryClassifier, self).__init__()

        self.layers = nn.ModuleList()
        
        # First hidden layer
        self.layers.append(nn.Linear(input_dim, hidden_dims[0]))

        # Remaining hidden layers
        for i in range(9):
            self.layers.append(nn.Linear(hidden_dims[i], hidden_dims[i + 1]))

        # Output layer
        self.layers.append(nn.Linear(hidden_dims[-1], 1))

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.relu(layer(x))
        x = self.sigmoid(self.layers[-1](x))
        return x


# Instantiate and train the model
model = BinaryClassifier(X_train_tensor.shape[1], [100, 100, 100, 100, 100, 100, 100, 100, 100])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print('Epoch: {} | Loss: {}'.format(epoch, loss.item()))

# Load the test data
test_data = pd.read_csv('data/test_final.csv')
test_ids = test_data['Id']
test_data.drop(columns=['Id'], inplace=True)

# Normalize the test features and handle missing values
X_test = X_test.replace(-999, np.nan)
X_test = X_test.fillna(test.mean())

# Convert to PyTorch tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Evaluate the model and get probabilities
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probabilities = outputs.squeeze().numpy()

# Create a DataFrame and save to CSV
results_df = pd.DataFrame({
    'Id': test_ids,
    'Y': probabilities
})
results_df.to_csv('neural_network_test_outputs.csv', index=False)

Epoch [1/4000], Loss: 0.6880
Epoch [2/4000], Loss: 0.6876
Epoch [3/4000], Loss: 0.6873
Epoch [4/4000], Loss: 0.6870
Epoch [5/4000], Loss: 0.6868
Epoch [6/4000], Loss: 0.6866
Epoch [7/4000], Loss: 0.6867
Epoch [8/4000], Loss: 0.6864
Epoch [9/4000], Loss: 0.6862
Epoch [10/4000], Loss: 0.6861
Epoch [11/4000], Loss: 0.6861
Epoch [12/4000], Loss: 0.6860
Epoch [13/4000], Loss: 0.6860
Epoch [14/4000], Loss: 0.6860
Epoch [15/4000], Loss: 0.6859
Epoch [16/4000], Loss: 0.6858
Epoch [17/4000], Loss: 0.6858
Epoch [18/4000], Loss: 0.6858
Epoch [19/4000], Loss: 0.6857
Epoch [20/4000], Loss: 0.6857
Epoch [21/4000], Loss: 0.6856
Epoch [22/4000], Loss: 0.6856
Epoch [23/4000], Loss: 0.6855
Epoch [24/4000], Loss: 0.6854
Epoch [25/4000], Loss: 0.6854
Epoch [26/4000], Loss: 0.6854
Epoch [27/4000], Loss: 0.6853
Epoch [28/4000], Loss: 0.6853
Epoch [29/4000], Loss: 0.6853
Epoch [30/4000], Loss: 0.6853
Epoch [31/4000], Loss: 0.6853
Epoch [32/4000], Loss: 0.6853
Epoch [33/4000], Loss: 0.6853
Epoch [34/4000], Lo

ValueError: array length 571 does not match index length 3854